In [9]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import warnings

import pandas as pd

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings("ignore")
from carla.data.api import data
import numpy as np
import torch
torch.cuda.is_available = lambda : False
import yaml
from seed_env import seed_my_session
from typing import Dict, List
from cent.data_specific import DataModels
from carla import Benchmark
import pandas as pd
from carla.recourse_methods import (
    CCHVAE,
    CEM,
    CRUD,
    FOCUS,
    ActionableRecourse,
    CausalRecourse,
    Clue,
    Dice,
    Face,
    FeatureTweak,
    GrowingSpheres,
    Revise,
    Wachter,
)
from carla.recourse_methods.catalog.causal_recourse import constraints, samplers
import carla.evaluation.catalog as evaluation_catalog
from cent.method import CEnt
from vae_benchmark import VAEBenchmark

seed_my_session()
def load_setup() -> Dict:
    with open("experimental_setup.yaml", "r") as f:
        setup_catalog = yaml.safe_load(f)
    return setup_catalog["recourse_methods"]

def print_conf(conf, d=4, d_iter=5):
    for k, v in conf.items():
        if isinstance(v, dict):
            print("{}{} : ".format(d * " ", str(k)))
            print_conf(v, d + d_iter)
        elif isinstance(v, list) and len(v) >= 1 and isinstance(v[0], dict):
            print("{}{} : ".format(d * " ", str(k)))
            for value in v:
                print_conf(value, d + d_iter)
        else:
            print("{}{} : {}".format(d * " ", k, v))
        
def get_resource_supported_backend(recourse_method, supported_backend_dict):
    '''
    Search in supported_backend for the recourse_method to know what backend is supported
    '''
    # supported_backend contains 'pytorch', 'tensorflow', 'xgboost', 'sklearn'
    # Check what backend contains the recourse_method
    suuported_backs = []
    for backend in supported_backend_dict:
        if recourse_method in supported_backend_dict[backend]:
            suuported_backs.append(backend)
    # If tensorflow and pytorch are in the list, keep only tensorflow
    if "tensorflow" in suuported_backs and "pytorch" in suuported_backs:
        # Remove pytorch from the list
        suuported_backs.remove("pytorch")
    # Similarly for xgboost and sklearn, keep xgboost
    if "xgboost" in suuported_backs and "sklearn" in suuported_backs:
        suuported_backs.remove("sklearn")
    #TODO: Keep both, but temp return only first
    return suuported_backs[0]

def intialialize_recourse_method(method, hyperparams, mlmodel, data_models):
    # TODO restrict data to training only
    if method == "cchvae":
        hyperparams["data_name"] = data_name
        hyperparams["vae_params"]["layers"] = [
            len(mlmodel.feature_input_order)
        ] + hyperparams["vae_params"]["layers"]
        return CCHVAE(mlmodel, hyperparams)
    elif "cem" in method:
        hyperparams["data_name"] = data_name
        raise ValueError("Session Methods not supported yet")
        #return CEM(sess, mlmodel, hyperparams)
    elif method == "clue":
        hyperparams["data_name"] = data_name
        return Clue(mlmodel.data, mlmodel, hyperparams)
    elif method == "cruds":
        hyperparams["data_name"] = data_name
        # variable input layer dimension is first time here available
        hyperparams["vae_params"]["layers"] = [
            len(mlmodel.feature_input_order)
        ] + hyperparams["vae_params"]["layers"]
        return CRUD(mlmodel, hyperparams)
    elif method == "dice":
        return Dice(mlmodel, hyperparams)
    elif "face" in method:
        return Face(mlmodel, hyperparams)
    elif method == "growing_spheres":
        return GrowingSpheres(mlmodel)
    elif method == "revise":
        hyperparams["data_name"] = data_name
        # variable input layer dimension is first time here available
        hyperparams["vae_params"]["layers"] = [
            len(mlmodel.feature_input_order)
        ] + hyperparams["vae_params"]["layers"]
        return Revise(mlmodel, mlmodel.data, hyperparams)
    elif "wachter" in method:
        return Wachter(mlmodel, hyperparams)
    elif "causal_recourse" in method:
        return CausalRecourse(mlmodel, hyperparams)
    elif "focus" in method:
        hyperparams = {'optimizer': 'adam', 'lr': 0.001, 'n_class': 2, 'n_iter': 1000, 'sigma': 1.0, 'temperature': 1.0, 'distance_weight': 0.01, 'distance_func': 'l1'}
        return FOCUS(mlmodel, hyperparams)
    elif "feature_tweak" in method:
        return FOCUS(mlmodel, hyperparams)
    elif "cent" in method:
        min_entries_per_label = int(data_models.trainData.df.shape[0]*0.02)
        if min_entries_per_label<900:
            min_entries_per_label = 900
        hpr = {"data_name": "data_name","n_search_samples": 300,"p_norm": 1,"step": 0.1,"max_iter": 10,"clamp": True,
                "binary_cat_features": True,
                "myvae_params": {
                    'input_dim': len(mlmodel.feature_input_order),
                    'kld_weight': 0.00025,
                    'layers': layers,
                    'latent_dim': latent_dim,
                    'hidden_activation': 'relu',
                    'dropout': 0.2,
                    'batch_norm': True,
                    'batch_size': 128,
                    'epochs': 1,
                    'learning_rate': 0.001,
                    'weight_decay': 0.0,
                    'cuda': False,
                    'verbose': True,
                    'train': True,
                    'save_dir': './vae_model/',
                },
                "tree_params": {
                    "min_entries_per_label": min_entries_per_label,
                    "grid_search_jobs": -1,
                    "min_weight_gini": 100, # set to 0.5 since here both class have same prob,
                    "max_search" : 50,
                    "grid_search": {"cv": 1,"splitter": ["best"],"criterion": ["gini"],"max_depth": [3,4,5,6,7,8,9,10],
                                    "min_samples_split": [1.0,2,3],"min_samples_leaf": [1,2,3],
                                    "max_features": [0.4, 0.6, 0.8],
                                    }
                }
          }
        print_conf(hpr)
        return CEnt(data_models.trainData, mlmodel, hpr, data_catalog= data_models.new_catalog_n)

    else:
        raise ValueError("Recourse method not known  {}".format(method))


setup_catalog = load_setup()

# data_names = ['adult', 'compas', 'give_me_some_credit', 'heloc']
supported_backend_dict = {'pytorch': ["cchvae", "clue", "cruds", "dice", "face", 'growing_spheres',"revise" 'wachter', 
                                    'causal_recourse','actionable_recourse'],
                        'tensorflow': ['cem', 'dice', 'face', 'growing_spheres', 'causal_recourse','actionable_recourse','cent'],
                        'sklearn': ['feature_tweak','focus'],
                        'xgboost': ['feature_tweak','focus']}


# VAE distance in benchmarking                                                  DONE
# Github migration                                                              DONE
# clue, dice, face, growing_spheres, [focus, cem,] crude, wama tayasar          ~DONE
# VAE latent representation layer size according to data columns                ~DONE
# VAE constraint
# VAE encodings distance in benchmarking                                        DONE
# Implement our working version of VAE (tested on MNIST with 2 neurons)         DONE
# Fix Best Metric in our version of VAE (load the best instead of using latest) DONE
# Save Benchmark results per row, factuals, counterfactuals                     DONE
# Save Tree results per row (Some time is added for inf)                        DONE
#       For (3124, 15) nearest neighbors, we have the following timings: 
#               1. DTree fitting per row: 11.6 ms ± 1.72 ms per run
#               2. Scoring (Inference+Score): 3.9 ms ± 126 µs per run



FACTUAL_NUMBER = 20

data_names = ['adult','compas', 'give_me_some_credit', 'heloc']

recourse_methods = ['cent','dice','growing_spheres','clue','causal_recourse',
                    'cchvae','cruds','focus','actionable_recourse',
                    'cem','revisewachter','face','feature_tweak']

NOTWORKING = [] # ['causal_recourse','focus'] # NOTWORKING
TESTEDSUCCESSFULLY = ['clue','dice','cent','cchvae'] # ALREADY TESTED


data_names = ['adult','heloc']

recourse_methods = ['cent','dice','growing_spheres']


# Define Output Directory
OUT_DIR = "./outputs/"
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

print(recourse_methods)


['cote', 'dice', 'growing_spheres']


In [3]:
data_name = data_names[0]
OUT_DIR_DATA = os.path.join(OUT_DIR, data_name)
if not os.path.exists(OUT_DIR_DATA):
    os.makedirs(OUT_DIR_DATA)

OUT_DIR_DATA_BENCH_CSVS = os.path.join(OUT_DIR_DATA, 'bench_csvs')
if not os.path.exists(OUT_DIR_DATA_BENCH_CSVS):
    os.makedirs(OUT_DIR_DATA_BENCH_CSVS)

# Load dataset and necessary models
data_models = DataModels(data_name = data_name,
                            factuals_length = FACTUAL_NUMBER,
                            out_dir = OUT_DIR_DATA)

# Load VAE
print("Starting VAE for benchmarking")
# Get an ann tensorflow model as temp just to get some hyperparams
temp_model = data_models.models_zoo['ann']['tensorflow']

if len(temp_model.feature_input_order) > 500:
    layers = [500, 250]
    latent_dim = 32
elif len(temp_model.feature_input_order) > 100:
    layers = [100, 50]
    latent_dim = 24
elif len(temp_model.feature_input_order) > 50:
    layers = [50, 25]
    latent_dim = 16
elif len(temp_model.feature_input_order) > 20:
    layers = [25, 16]
    latent_dim = 12
elif len(temp_model.feature_input_order) > 10:
    layers = [25]
    latent_dim = 8
else:
    layers = [16]
    latent_dim = 7
xxmutables = []
for i in range(len(temp_model.feature_input_order)):
    xxmutables.append(True)
xxmutables = np.array(xxmutables)
vae_parms = { 
    "myvae_params": {
        'input_dim': len(temp_model.feature_input_order),
        'kld_weight': 0.00025,
        'layers': layers,
        'latent_dim': latent_dim,
        'hidden_activation': 'relu',
        'dropout': 0.2,
        'batch_norm': True,
        'batch_size': 32,
        'epochs': 20,
        'learning_rate': 0.001,
        'weight_decay': 0.0,
        'cuda': False,
        'verbose': True,
        'train': True,
        'save_dir': './vae_model/',
    }
}
print_conf(vae_parms)

Loading models... --- logs will be saved to ./outputs/adult/models_logs.txt
[WARNING] From /home/mmansour/Music/AUB/Julia/TreeBased_Carla/carla/models/catalog/ANN_TF/model_ann.py:10: The name tf.log is deprecated. Please use tf.math.log instead.
 [deprecation_wrapper.py __getattr__]
[WARNING] From /home/mmansour/miniconda3/envs/xai/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.
 [deprecation_wrapper.py __getattr__]
Starting VAE for benchmarking
    myvae_params : 
         input_dim : 13
         kld_weight : 0.00025
         layers : [25]
         latent_dim : 8
         hidden_activation : relu
         dropout : 0.2
         batch_norm : True
         batch_size : 32
         epochs : 20
         learning_rate : 0.001
         weight_decay : 0.0
         cuda : False
         verbose : True
         train : True
         save_dir : ./vae_model/


In [5]:
metrics_scores = []
# Define csv file to store results
csv_file = os.path.join(OUT_DIR_DATA, 'benchmark_results.csv')
recourse_method = 'cent'
# Define Checkers
test_checks = {'Resource_Method':[], 'Success_Boolean': [], 'model_type':[],'Details':[]}
check_csv = os.path.join(OUT_DIR_DATA, 'checks.csv')
# Loop over recourse methods
supported_backend = get_resource_supported_backend(recourse_method, supported_backend_dict)
if supported_backend in ['tensorflow', 'pytorch']:
    supported_types = ['linear', 'ann']
else:
    supported_types = ['forest']
supported_type = 'linear'

In [6]:
model_temp = data_models.models_zoo[supported_type][supported_backend]
if recourse_method in setup_catalog:
    if 'hyperparams' in setup_catalog[recourse_method]:
        hyperpars = setup_catalog[recourse_method]['hyperparams']
else:
    hyperpars = {}

In [8]:
from copy import deepcopy
# copy data_models.trainData
copy_dataset = deepcopy(data_models.trainData)

In [10]:
rcmethod = intialialize_recourse_method(recourse_method, hyperpars, model_temp, data_models)

    data_name : data_name
    n_search_samples : 300
    p_norm : 1
    step : 0.1
    max_iter : 10
    clamp : True
    binary_cat_features : True
    myvae_params : 
         input_dim : 13
         kld_weight : 0.00025
         layers : [25]
         latent_dim : 8
         hidden_activation : relu
         dropout : 0.2
         batch_norm : True
         batch_size : 128
         epochs : 1
         learning_rate : 0.001
         weight_decay : 0.0
         cuda : False
         verbose : True
         train : True
         save_dir : ./vae_model/
    tree_params : 
         min_entries_per_label : 975
         grid_search_jobs : -1
         min_weight_gini : 100
         max_search : 50
         grid_search : 
              cv : 1
              splitter : ['best']
              criterion : ['gini']
              max_depth : [3, 4, 5, 6, 7, 8, 9, 10]
              min_samples_split : [1.0, 2, 3]
              min_samples_leaf : [1, 2, 3]
              max_features : [0.4, 0.6, 0.

'tensorflow'

In [21]:
# Get accuracy score
# Import get_accuracy_score
from sklearn.metrics import accuracy_score
y_true = copy_dataset.df[model_temp.data.target].values
y_pred = rcmethod.dataset[model_temp.data.target].values
accuracy_score(y_true, y_pred)

0.8227671614860986

In [16]:
setup_catalog = load_setup()

# data_names = ['adult', 'compas', 'give_me_some_credit', 'heloc']
supported_backend_dict = {'pytorch': ["cchvae", "clue", "cruds", "dice", "face", 'growing_spheres',"revise" 'wachter', 
                                    'causal_recourse','actionable_recourse'],
                        'tensorflow': ['cem', 'dice', 'face', 'growing_spheres', 'causal_recourse','actionable_recourse','cent'],
                        'sklearn': ['feature_tweak','focus'],
                        'xgboost': ['feature_tweak','focus','cent']}
FACTUAL_NUMBER = 20

data_names = ['adult','compas', 'give_me_some_credit', 'heloc']

recourse_methods = ['cent','dice','growing_spheres','clue','causal_recourse',
                    'cchvae','cruds','focus','actionable_recourse',
                    'cem','revisewachter','face','feature_tweak']

NOTWORKING = [] # ['causal_recourse','focus'] # NOTWORKING
TESTEDSUCCESSFULLY = ['clue','dice','cent','cchvae'] # ALREADY TESTED


data_names = ['adult','heloc']

recourse_methods = ['cem','cent','dice','growing_spheres']


# Define Output Directory
OUT_DIR = "./outputs/"
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

print(recourse_methods)
data_name = data_names[1]
print(data_name)

['cem', 'cote', 'dice', 'growing_spheres']
heloc


In [17]:
OUT_DIR_DATA = os.path.join(OUT_DIR, data_name)
if not os.path.exists(OUT_DIR_DATA):
    os.makedirs(OUT_DIR_DATA)

OUT_DIR_DATA_BENCH_CSVS = os.path.join(OUT_DIR_DATA, 'bench_csvs')
if not os.path.exists(OUT_DIR_DATA_BENCH_CSVS):
    os.makedirs(OUT_DIR_DATA_BENCH_CSVS)

# Load dataset and necessary models
data_models = DataModels(data_name = data_name,
                            factuals_length = FACTUAL_NUMBER,
                            out_dir = OUT_DIR_DATA)


Loading models... --- logs will be saved to ./outputs/heloc/models_logs.txt


In [19]:
mlmodel = data_models.models_zoo['ann']['tensorflow']

In [20]:
hyperpars = setup_catalog['cem']['hyperparams']

In [25]:
from tensorflow import Graph, Session
ann_sess = Session()

[WARNING] From /tmp/ipykernel_77298/3545653047.py:1: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.
 [deprecation_wrapper.py __getattr__]


In [36]:
def intialialize_recourse_method(method, hyperparams, mlmodel, data_models, ann_sess_ = None):
    # TODO restrict data to training only
    if method == "cchvae":
        hyperparams["data_name"] = data_name
        hyperparams["vae_params"]["layers"] = [
            len(mlmodel.feature_input_order)
        ] + hyperparams["vae_params"]["layers"]
        return CCHVAE(mlmodel, hyperparams)
    elif "cem" in method:
        hyperparams["data_name"] = data_name
        return CEM(ann_sess_, mlmodel, hyperparams)
        #return CEM(sess, mlmodel, hyperparams)
    elif method == "clue":
        hyperparams["data_name"] = data_name
        return Clue(mlmodel.data, mlmodel, hyperparams)
    elif method == "cruds":
        hyperparams["data_name"] = data_name
        # variable input layer dimension is first time here available
        hyperparams["vae_params"]["layers"] = [
            len(mlmodel.feature_input_order)
        ] + hyperparams["vae_params"]["layers"]
        return CRUD(mlmodel, hyperparams)
    elif method == "dice":
        return Dice(mlmodel, hyperparams)
    elif "face" in method:
        return Face(mlmodel, hyperparams)
    elif method == "growing_spheres":
        return GrowingSpheres(mlmodel)
    elif method == "revise":
        hyperparams["data_name"] = data_name
        # variable input layer dimension is first time here available
        hyperparams["vae_params"]["layers"] = [
            len(mlmodel.feature_input_order)
        ] + hyperparams["vae_params"]["layers"]
        return Revise(mlmodel, mlmodel.data, hyperparams)
    elif "wachter" in method:
        return Wachter(mlmodel, hyperparams)
    elif "causal_recourse" in method:
        return CausalRecourse(mlmodel, hyperparams)
    elif "focus" in method:
        hyperparams = {'optimizer': 'adam', 'lr': 0.001, 'n_class': 2, 'n_iter': 1000, 'sigma': 1.0, 'temperature': 1.0, 'distance_weight': 0.01, 'distance_func': 'l1'}
        return FOCUS(mlmodel, hyperparams)
    elif "feature_tweak" in method:
        return FOCUS(mlmodel, hyperparams)
    elif "cent" in method:
        min_entries_per_label = int(data_models.trainData.df.shape[0]*0.02)
        if min_entries_per_label<900:
            min_entries_per_label = 900
        hpr = {"data_name": "data_name","n_search_samples": 300,"p_norm": 1,"step": 0.1,"max_iter": 10,"clamp": True,
                "binary_cat_features": True,
                "myvae_params": {
                    'input_dim': len(mlmodel.feature_input_order),
                    'kld_weight': 0.00025,
                    'layers': layers,
                    'latent_dim': latent_dim,
                    'hidden_activation': 'relu',
                    'dropout': 0.2,
                    'batch_norm': True,
                    'batch_size': 32,
                    'epochs': 10,
                    'learning_rate': 0.001,
                    'weight_decay': 0.0,
                    'cuda': False,
                    'verbose': True,
                    'train': True,
                    'save_dir': './vae_model/',
                },
                "tree_params": {
                    "min_entries_per_label": min_entries_per_label,
                    "grid_search_jobs": -1,
                    "min_weight_gini": 100, # set to 0.5 since here both class have same prob,
                    "max_search" : 100,
                    "grid_search": {"cv": 1,"splitter": ["best"],"criterion": ["gini"],"max_depth": [3,4,5,6,7,8,9,10],
                                    "min_samples_split": [1.0,2,3],"min_samples_leaf": [1,2,3],
                                    "max_features": [0.4, 0.6, 0.8],
                                    }
                }
          }
        print_conf(hpr)
        return CEnt(data_models.trainData, mlmodel, hpr, data_catalog= data_models.new_catalog_n)

    else:
        raise ValueError("Recourse method not known  {}".format(method))


rcmethod = intialialize_recourse_method('cem', hyperpars, mlmodel, data_models)

Train on 6909 samples, validate on 2962 samples
Epoch 1/5
6909/6909 [==============================] - 0s 41us/step - loss: 0.0945 - val_loss: 0.0419
Epoch 2/5
6909/6909 [==============================] - 0s 13us/step - loss: 0.0302 - val_loss: 0.0256
Epoch 3/5
6909/6909 [==============================] - 0s 14us/step - loss: 0.0245 - val_loss: 0.0218
Epoch 4/5
6909/6909 [==============================] - 0s 14us/step - loss: 0.0194 - val_loss: 0.0165
Epoch 5/5
6909/6909 [==============================] - 0s 12us/step - loss: 0.0157 - val_loss: 0.0142


In [9]:
layers = [25]
latent_dim = 6

min_entries_per_label = int(data_models.trainData.df.shape[0]*0.02)
if min_entries_per_label<900:
    min_entries_per_label = 900
hpr = {"data_name": "data_name","n_search_samples": 300,"p_norm": 1,"step": 0.1,"max_iter": 10,"clamp": True,
        "binary_cat_features": True,
        "myvae_params": {
            'input_dim': len(mlmodel.feature_input_order),
            'kld_weight': 0.00025,
            'layers': layers,
            'latent_dim': latent_dim,
            'hidden_activation': 'relu',
            'dropout': 0.2,
            'batch_norm': True,
            'batch_size': 128,
            'epochs': 1,
            'learning_rate': 0.001,
            'weight_decay': 0.0,
            'cuda': False,
            'verbose': True,
            'train': True,
            'save_dir': './vae_model/',
        },
        "tree_params": {
            "min_entries_per_label": min_entries_per_label,
            "grid_search_jobs": -1,
            "min_weight_gini": 100, # set to 0.5 since here both class have same prob,
            "max_search" : 100,
            "grid_search": {"cv": 1,"splitter": ["best"],"criterion": ["gini"],"max_depth": [3,4,5,6,7,8,9,10],
                            "min_samples_split": [1.0,2,3],"min_samples_leaf": [1,2,3],
                            "max_features": [0.4, 0.6, 0.8],
                            }
        }
    }
print_conf(hpr)
rcmethod = CEnt(data_models.trainData, mlmodel, hpr, data_catalog= data_models.new_catalog_n)

    data_name : data_name
    n_search_samples : 300
    p_norm : 1
    step : 0.1
    max_iter : 10
    clamp : True
    binary_cat_features : True
    myvae_params : 
         input_dim : 21
         kld_weight : 0.00025
         layers : [25]
         latent_dim : 6
         hidden_activation : relu
         dropout : 0.2
         batch_norm : True
         batch_size : 128
         epochs : 1
         learning_rate : 0.001
         weight_decay : 0.0
         cuda : False
         verbose : True
         train : True
         save_dir : ./vae_model/
    tree_params : 
         min_entries_per_label : 900
         grid_search_jobs : -1
         min_weight_gini : 100
         max_search : 100
         grid_search : 
              cv : 1
              splitter : ['best']
              criterion : ['gini']
              max_depth : [3, 4, 5, 6, 7, 8, 9, 10]
              min_samples_split : [1.0, 2, 3]
              min_samples_leaf : [1, 2, 3]
              max_features : [0.4, 0.6, 0

In [32]:
mlmodel = data_models.models_zoo['ann']['tensorflow']

In [35]:
graph = Graph()
with graph.as_default():
    ann_sess = Session()
    with ann_sess.as_default():
        rcmethod = intialialize_recourse_method('cem', hyperpars, mlmodel, data_models, ann_sess_ = ann_sess)
        benchmark = Benchmark(mlmodel, rcmethod,  data_models.factuals.copy().reset_index(drop=True))

Train on 6909 samples, validate on 2962 samples
Epoch 1/5
6909/6909 [==============================] - 0s 25us/step - loss: 0.0889 - val_loss: 0.0390
Epoch 2/5
6909/6909 [==============================] - 0s 12us/step - loss: 0.0288 - val_loss: 0.0224
Epoch 3/5
6909/6909 [==============================] - 0s 11us/step - loss: 0.0201 - val_loss: 0.0186
Epoch 4/5
6909/6909 [==============================] - 0s 12us/step - loss: 0.0177 - val_loss: 0.0167
Epoch 5/5
6909/6909 [==============================] - 0s 14us/step - loss: 0.0159 - val_loss: 0.0145


ValueError: Tensor("dense_1_1/kernel/Read/ReadVariableOp:0", shape=(21, 13), dtype=float32) must be from the same graph as Tensor("Variable_1/read:0", shape=(1, 21), dtype=float32).

In [12]:
measures = [
                    evaluation_catalog.YNN(benchmark.mlmodel, {"y": 5, "cf_label": 1}),
                    evaluation_catalog.Distance(benchmark.mlmodel),
                    evaluation_catalog.SuccessRate(),
                    evaluation_catalog.Redundancy(benchmark.mlmodel, {"cf_label": 1}),
                    evaluation_catalog.ConstraintViolation(benchmark.mlmodel),
                    evaluation_catalog.AvgTime({"time": benchmark.timer})                ]

In [13]:
resource_bench = benchmark.run_benchmark(measures = measures)

In [15]:
resource_bench.mean()

y-Nearest-Neighbours    0.457143
L0_distance             2.571429
L1_distance             0.253065
L2_distance             0.050363
Linf_distance           0.151970
Success_Rate            0.700000
Redundancy              1.357143
Constraint_Violation    0.000000
avg_time                0.181596
dtype: float64